In [1]:
import os
import random
import numpy as np
import requests
import vk_api
import urllib.request
from vk_api import VkUpload
from vk_api.longpoll import VkLongPoll, VkEventType, VkLongpollMode

In [2]:
token_=''
url_begin='https://psv4.userapi.com/'

**Варианты приветствий**

In [3]:
#TODO: оставить только нижний регистр
welcome=['привет', 'Привет','здарова', 'прив','дарова','Здарова', 'Халло', 'халло','Хелло', 'хэллоу','Хэллоу',
         'хелоу','хело']
how_are_you=['Как дела?','как дела?', 'как жизнь?', 'дела','дела?','как поживаешь?']
agromsg=['Тупой','тупой','Ты дурак', 'Дурак', 'дурак', 'Дебил', 'дебил', 'дурачок', 'конченный', 'Конченный', 
         'глупый', 'Ты глупый', 'ты глупый','Баран', 'баран', 'тупой бот', 'Тупой бот', 
         'бяка', 'Бяка', 'Гадина', 'гадина', 'даун', 'Даун', 'Дэбил', 'дэбил', 'обормот', 
         'Обормот', 'Капец ты тупой','капец ты тупой', 'Лох', 
         'лох', 'критин', 'Критин', 'Мудак', 'мудак']
agrootv=['Кто обзывается, тот тем и называется!', 'Если ты так будешь общаться, то я тебя побью',
         'Фу, ужасное воспитание!','Было два стула, на какой сам сядешь, на какой бота посадишь', 
         'Я все маме расскажу!', 'За тобой уже выехали', 'Если ты меня еще раз оскорбишь,то тебе крышка',
         'За школой в 5']
randomanser=['Здорово', 'С тобой интересно общаться', 'Мне кажется, что неплохо', 'Ого', 'И вправду круто', 
             'Очень даже', 'Мне нравится', 'Пипееец', 'Оу маиин', 'Что? Серьезно? Четка!','В принципе неплохо!', 
             'Понятно, что-то еще?', 'Это действительно интересно!','Ого, расскажешь еще что-нибудь?',
             'Не, давай общаться нормально!']
whatDoUDo=['Что делаешь', 'делаешь']
drugs=['наркотики', 'алкоголь', 'пиво', 'водка', 'кит', 
       'смерть', 'умри','сигареты', 'умрешь', 'сдохни', 'убью']
ploho=['Это очень плохо', 'Не пиши такое!', 'Нельзя говорить подобные вещи!', 
       'Если ты еще раз подобное скажешь,то к тебе приедут определенные службы', 
       'Так,не стоит говорить такое, иначе позвоню твоей маме!', 
       'Слушай я все понимаю, но не нужно об этом говорить и даже думать!', 
       'Тебе подсказать телефон доверия?']
helper='На данный момент я умею отвечать на твои сообщения разного характера (будь то хорошие или плохие).\n В будущем я научусь редактировать аудиозаписи и накладывать туда разные голоса:)'

## Обработка `longpoll` запросов
**Два вспомогательных класса-хендлера для обработки входящих запросов**

In [4]:
#@DaniloZZZ 
class Longpoll():
    def __init__(self, upd_checker,
                 params_modifier=lambda x,u: x):
        """
        :param upd_checker:
            a function that returns if
            there was update
            :inp: dict
            :returns: Bool
        :param params_modifier:
            a function that maps previous params
            every time request done and provides
            info if there was updates
            :inp: (dict,bool)
            :returns: dict
        """
        self.func= upd_checker
        self.param_next= params_modifier

    def event_emmitter(self,addr,params={}):
        while True:
            r = requests.get(addr,params)
            js = r.json()
            if r.status_code==200:
                if (self.func(js)):
                    yield js
                    print('upd',params)
                    self.param_next(params,True)
                else:
                    print('no updates, listening more...')
                    self.param_next(params,False)
            else:
                raise Exception('longpoller error',
                                r.status_code,js,
                                'params:',params
                                )

class VKBot:
    ep = 'https://api.vk.com/method'
    version = '5.80'

    def __init__(self,token,group):
        self.token = token
        self.group = group
        self.get_longpoll_server(group)
        #self.check_longpoll()
        self.poller = Longpoll(
            lambda x: len(x.get('updates',[]))>0,
            params_modifier=
            lambda x,u: x.update({'ts':x.get('ts',0)+1})\
            if u else x
        )

    def start_polling(self,**args):
        addr = self.longpoll_server
        params = {
            'act':'a_check',
            'key':self.longpoll_key,
            'ts':self.ts
        }
        for event in self.poller.event_emmitter(addr,params):
            try:
                #print('$$$$',event['updates'][0]['object']['attachments'],'$$$$')
                t = random.randint(0,100) 
                url_=event['updates'][0]['object']['attachments'][0]['doc']['preview']['audio_msg']['link_ogg']
                name_ = 'VoiceAnswers/'+str(t)+'.ogg'
                print(type(url_), url_)
                print(type(name_),name_)
                #print('!!!!!!!!!!',url_,'!!!!!!!!!!!!!!')
                audio = urllib.request.urlretrieve(filename=name_, 
                                                   url=url_)            
                print(audio)
                return audio
            except KeyError:
                pass
            

    def send_message(self,chat_id,text,**args):
        params = {
                'peer_id':chat_id,
                'message':text,
            }
        mk = args.get('reply_markup')
        print(mk.get_json())
        if mk:
            k =  mk.get_json()
            params.update({
               # 'keyboard':mk.get_json()
                'keyboard':k
            })

        r = self.make_request(
            'messages.send',
            params
        )
        return r


    def set_message_handler(self,clb):
        self.message_handler =  clb
    def set_callback_handler(self,clb):
        self.callback_handler = clb

    def get_longpoll_server(self,group):
        print("getting longpoll server addr")
        s = self.make_request('groups.getLongPollServer',
                          {
                            'group_id':group
                          }
                         )
        server,key = s.get('server'),s.get('key')
        # Assign the props to self
        if (server and key):
            self.longpoll_server, self.longpoll_key = server,key
            self.ts = s.get('ts')
        else:
            raise Exception("Vk returned no Longpoll servers")

    def check_longpoll(self):
        r = requests.get(
            self.longpoll_server,
            {
                'act':'a_check',
                'key':self.longpoll_key,
                'ts':1,
                'wait':2
            }
        )
        js = r.json()
        print('Checked longpoll',js)
        return js.get('response')

    def make_request(self,method_name,method_params):
        ep = 'https://api.vk.com/method/'
        ep += method_name
        params = {
            'access_token':self.token,
            'v':self.version
        }
        params.update(method_params)
        r = requests.get(ep,params=params)
        js = r.json()
        print(r)
        print(r.text)
        return js.get('response')


Инициализация токена

In [5]:
group_id = '168509549'
bot = VKBot(token_,group_id)

getting longpoll server addr
<Response [200]>
{"response":{"key":"2fe5333e5040deebdf57fca8ce43fe79d03ddbdd","server":"https:\/\/lp.vk.com\/wh168509549","ts":875}}


# Функции-обработчики сообщений (текстовых и аттачей)

**Эта функция получает на вход сообщение от пользователя, обрабатывает его и посылает пользователю ответ. На вход подается очередное слово на обработку, а также количество слов в сообщении.**

In [6]:
def handle_text_msg(text_from_usr_,length,vk,name):
    flag=0
    tmp=0
    for i in range(len(text_from_usr_)):
        text_from_usr=text_from_usr_[i]
        
        if text_from_usr in welcome:
            print('==welocome==')
            flag += 1
            return 'привет, '+name
        
                    
        if text_from_usr in whatDoUDo:
            print('==WhatDoUDo==')
            flag += 1
            return 'с тобой говорю, а ты?'
                    
        if text_from_usr in drugs:
            print('==Drugs==')
            flag += 1
            return str(random.choice(ploho))

        if text_from_usr in ['help', 'Help']:
            print('==Help==')
            flag += 1
            return helper

        if text_from_usr in agromsg:
            r='сам ты '+text_from_usr
            agrootv.append(r)
            lol1=random.choice(agrootv)
            agrootv.remove(r)
            print('==Evil==')
            flag += 1
            return lol1


        if text_from_usr in how_are_you:
            print('==HowAreYou==')
            flag += 1
            return 'неплохо, спасибо! А как твои дела?'
        
    if (length==1) and flag==0:
        flag += 1
        print('==AnyText==')
        return random.choice(randomanser)
    else:
            return random.choice(randomanser)
                    


    '''
    vk.messages.send(user_id=event.user_id, 
                         message='Я уже научился принимать и обрабатывать голосовые сообщения* '
                    )
                    image_url = response.get('Image')
                    image = vk.get(image_url, stream=True)
                    photo = upload.photo_messages(photos=image.raw)[0]

                    attachments.append(
                        'photo{}_{}'.format(photo['owner_id'], photo['id'])
                    )

                    vk.messages.send(
                        user_id=event.user_id,
                        attachment=','.join(attachments),
                        message=text
                        )
                    print('ok')
                    print('****',event.attachments,'****')    
                        
                    print('==Attach recieved==')
                    print(event.attachments)
   
    '''

In [7]:
def handle_attach (event, vk):
    try:
        if not(event.attachments['attach1_kind']=='audiomsg'):
            vk.messages.send(
                user_id=event.user_id,
                message='Пришли-ка лучше мне голосовое сообщение, дружок-пирожок'
                )
            return 1
    except KeyError:
        vk.messages.send(user_id=event.user_id,
            message='Пришли-ка лучше мне голосовое сообщение, дружок-пирожок'
            )
        return 2

   
    vk.messages.send(user_id=event.user_id, 
        message='Я уже научился принимать и обрабатывать голосовые сообщения, скоро я научусь отправлять их в ответ. Немного терпения,пупсик :* '
        )
    tmp = random.randint(0,100)
    name='file'+str(tmp)+'.ogg'
    url_=bot.start_polling()
    #audio = urllib.request.urlretrieve(filename='url[14]'+'.ogg',url=url_)
    return 1          

## Основная функция работы программы

In [8]:
def main():
    #Запускаем сессию и авторизируем бота через токен
    session = requests.Session()
    vk_session = vk_api.VkApi(token=token_)
    
    print('Initialized vk_session')
    vk = vk_session.get_api()
    
    #будем использовать longpoll api для онлайн-ответов от бота
    longpoll = VkLongPoll(vk_session,preload_messages=True)
    #ожидаем (listen) очередное событие event от пользователя
    i = 0
    
    '''
        Это основной цикл, где бот ожидает очередного 
        сообщения от пользователя и обрабатывает его 
    '''
    
    for event in longpoll.listen():
        i += 1
        #если сообщение -- новое и прислано нашему боту, то обрабатываем его:
        if event.type == VkEventType.MESSAGE_NEW and event.to_me:
            s=agrootv
            
            '''
                Посмотрим, от кого пришло НОВОЕ сообщение и 
                найдем информацию о пользователе.
                Разбираем сначала текстовое сообщение, затем -- прикрепленный аттач.
            
            '''
            text_flag = 0
            usr_info = vk_session.method('users.get', {'user_ids': event.user_id, 'fields': 'photo_50, city, verified'})
            name = usr_info[0]['first_name']
            print(i,' Сообщение от ''{}: "{}"'.format(usr_info[0]['last_name'], event.text), end=' ')
            text_from_usr=event.text
            text_from_usr_=text_from_usr.lower().split()
            for i in range(len(text_from_usr_)):
                text_from_usr=text_from_usr_[i]
                text_flag+=1
                

            if event.text:
                msg = handle_text_msg(text_from_usr_=text_from_usr_,
                                    length=len(text_from_usr_),
                                    vk = vk_session.get_api(),
                                    name=name
                                   )
                vk.messages.send(
                                user_id=event.user_id,
                                message=msg
                            )
                
                
            else:
                if len(event.text)==0:
                    print('recieved audio')
                    audio = handle_attach(event,vk = vk_session.get_api())
            print('completed')


In [9]:
if __name__ == '__main__':
    main()

Initialized vk_session


KeyboardInterrupt: 